### imports 

In [1]:
import os
from os import path
import configparser
from github import Github
import pandas as pd
import numpy
import datetime
from datetime import datetime
from datetime import timezone
import requests
from requests.adapters import HTTPAdapter, Retry

In [2]:
import githubanalysis.processing.repo_name_clean as name_clean
import githubanalysis.processing.get_repo_connection as ghconnect
import githubanalysis.processing.setup_github_auth as ghauth
import githubanalysis.processing.get_all_pages_issues as getallissues
import githubanalysis.analysis.calc_days_since_repo_creation as dayssince
import githubanalysis.processing.summarise_repo_stats as summarisestats

### access token 

In [3]:
# set up github access token with github package: 

config = configparser.ConfigParser()
config.read('../config.cfg')
config.sections()

access_token = config['ACCESS']['token']
g = Github(access_token) 

### get stats for 1 repo  

In [4]:
config_path = '../config.cfg'
#config_path = '../../githubanalysis/config.cfg'

In [5]:
summarisestats.summarise_repo_stats(repo_name='ramses-antibiotics/snomedizer', config_path='../../githubanalysis/config.cfg', per_pg=1, verbose=True)

GH link opened
API response for getting total commits: <Response [200]>
API response for getting total commits in year: <Response [200]>
API response for getting PRs info: <Response [200]>


OSError: [Errno Config file does not exist at path:] githubanalysis/config.cfg